In [1]:
param_market = 'US'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[i, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE': 
                    url = 'https://www.ebay.de/itm/' + input_.loc[i, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU': 
                    url = 'https://www.ebay.com.au/itm/' + input_.loc[i, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =
                
                js = json.loads(resp.split('''$MC=(window.$MC||[]).concat(''')[1].split(''']}]})</script></body></html>''')[0] + ']}]}')

                # = = = = = = = = = = = = = = =
                
                market_id = js['o']['g']['marketplaceId'].strip()

                # = = = = = = = = = = = = = = =
                
                country = js['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = js['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = js['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = js['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if item_id != input_.loc[i, 'Item_Number']:
                    continue
                
                # = = = = = = = = = = = = = = =
                
                specification = js['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US' or param_market == 'AU':
                    if 'VEHICLE_SELECTION' in js['o']['w'][0][2]['model']['modules']:
                        category_id = js['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                        print(js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                elif param_market == 'DE':
                    if 'MOTORS_TAB' in js['o']['w'][0][2]['model']['modules']:
                        category_id = js['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        print(js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                        
                # = = = = = = = = = = = = = = =
                
                title = js['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in js['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = js['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''

                # = = = = = = = = = = = = = = =
                
                if 'binModel' in js['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = js['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in js['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = js['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()
                    
                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in js['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = js['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if param_market == 'US' or param_market == 'AU':
                            if 'available' in text['text']:
                                available = text['text'].strip()
                            elif 'sold' in text['text']:
                                sold = text['text'].strip()
                        elif param_market == 'DE':
                            if 'verfügbar' in text['text']:
                                available = text['text'].strip()
                            elif 'verkauft' in text['text']:
                                sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': '',
                                         'Item_Number': input_.loc[i, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                if param_market == 'US' or param_market == 'AU':
                    for j, name_value in zip(range(len(specification)), specification):
                        label = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                        if j == 0 and label == 'Condition':
                            df_temp['Condition'] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                        else:
                            df_temp[label.lower()] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()
                elif param_market == 'DE':
                    for j, name_value in zip(range(len(specification)), specification):
                        label = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                        if j == 0 and name_value == 'condition':
                            df_temp['Artikelzustand'] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                        else:
                            df_temp[label.lower()] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'No.': int(input_.loc[i, 'No.'])}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Item_Number'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True])
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True])
    output_error.to_excel('./item_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：17

176244763774  <->  [ok] - 剩余数量：0 - 当前时间：09:24:09
176246556457  <->  [ok] - 剩余数量：0 - 当前时间：09:24:09
This part is compatible with 521 vehicle(s)
185449374707  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
This part is compatible with 740 vehicle(s)
175283365190  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
This part is compatible with 1320 vehicle(s)
174887319475  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
This part is compatible with 1348 vehicle(s)
175338385369  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
This part is compatible with 2400 vehicle(s)
185542873237  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
186304798481  <->  [ok] - 剩余数量：0 - 当前时间：09:24:10
This part is compatible with 1541 vehicle(s)
174336571599  <->  [ok] - 剩余数量：0 - 当前时间：09:24:11
186306186729  <->  [ok] - 剩余数量：0 - 当前时间：09:24:11
This part is compatible with 1523 vehicle(s)
175350812271  <->  [ok] - 剩余数量：0 - 当前时间：09:24:11
176246552208  <->  [ok] - 剩余数量：0 - 当前时间：09:24:11
This part is compatible with 7 vehicle(s)
175372769695  <->  [ok] - 剩余数量：0 - 当前时间：09:24:11
